In [1]:
%%time
import warnings
warnings.filterwarnings('ignore')
from functools import partial
from collections import defaultdict, namedtuple
import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from itertools import product

from video699.screen.semantic_segmentation.fastai_detector import *
from video699.screen.semantic_segmentation.common import *
from video699.screen.semantic_segmentation.postprocessing import *
from video699.screen.semantic_segmentation.evaluation import *

CPU times: user 7.38 s, sys: 864 ms, total: 8.24 s
Wall time: 5.05 s


In [2]:
resize_factor = [2, 8]
frozen_epochs = [2, 5, 6]
unfrozen_epochs = [3, 4, 7]
base_lower_bound = [5, 7, 10, 15]
erosion_dilation_kernel_size = [20, 50, 80, 150]
ratio_split_lower_bound = [0.3, 0.4, 0.5, 0.7, 0.8, 0.9]

In [4]:
train = list(product(resize_factor, frozen_epochs, unfrozen_epochs))
train_names = ['resize_factor', 'frozen_epochs', 'unfrozen_epochs']

post_processing = list(product(base_lower_bound, erosion_dilation_kernel_size, ratio_split_lower_bound))
post_processing_names = ['base_lower_bound', 'erosion_dilation_kernel_size', 'ratio_split_lower_bound']

all_lectures = [video.filename for video in ALL_VIDEOS]
all_frames = [frame for video in ALL_VIDEOS for frame in video]
all_frames_grouped_by_videos = {video.filename: [frame for frame in video] for video in ALL_VIDEOS}

detector = FastAIScreenDetector()
actual_detector = AnnotatedSampledVideoScreenDetector()

In [5]:
def filtered_by(name, used):
    return any([lecture in str(name) for lecture in used]) and 'frame' in str(name)

def split_by(name, validation):
    return any([lecture in str(name) for lecture in validation])

In [9]:
def model_selection(lectures, train_names, post_processing_names, default_filtered_by, default_split_by):
    def make_splits(lectures):
        Split = namedtuple('Split', ['train', 'valid'])
        kf = KFold(n_splits=5, shuffle=True, random_state=123)
        splits = {}
        for j, split in enumerate(kf.split(lectures)):    
            train_lectures = [lectures[index] for index in split[0]]
            valid_lectures = [lectures[index] for index in split[1]]
            splits[j] = Split(train=train_lectures, valid=valid_lectures)
        return splits

    splits = make_splits(all_lectures)
    df_all = pd.DataFrame(columns=train_names + post_processing_names + ['iou', 'wrong_count', 'kfold_split'])

    for train_values in tqdm(train):
        resize_factor, frozen_epochs, unfrozen_epochs = train_values
        CONFIGURATION['resize_factor'] = str(resize_factor)
        CONFIGURATION['frozen_epochs'] = str(frozen_epochs)
        CONFIGURATION['unfrozen_epochs'] = str(unfrozen_epochs)

        for j in splits.keys():
            filtered_by = partial(default_filtered_by, used=splits[j].train + splits[j].valid)
            split_by = partial(default_split_by, validation=splits[j].valid)

            detector = FastAIScreenDetector(filtered_by=filtered_by, valid_func=split_by)
            detector.train()

            valid_frames = [frame for frame in all_frames if split_by(frame.pathname)]
            actuals = [actual_detector.detect(frame) for frame in valid_frames]
            sem_preds = detector.semantic_segmentation_batch(valid_frames)

            for post_processing_values in post_processing:    
                preds = detector.post_processing_batch(sem_preds, valid_frames, **dict(zip(post_processing_names, post_processing_values)))
                wrong_count, ious, _ = evaluate(actuals, preds)

                iou_score = np.nanmean(ious)
                wrong_count = len(wrong_count)
                df_all.loc[len(df_all)] = train_values + post_processing_values + (iou_score, wrong_count, j)
    return df_all

In [10]:
def convert_params(best_params):
    converted_params = []
    for i, par in enumerate(best_params):
        if par.is_integer():
            converted_params.append(int(par))
        elif isinstance(par, np.int64) or isinstance(par, np.float64):
            converted_params.append(par.item())
        else:
            converted_params.append(par)
    best_params = tuple(converted_params)
    
    return best_params

### Lecture-wise 5-fold cross validation

In [11]:
kf = KFold(n_splits=5, shuffle=True, random_state=123)
df_best_models = pd.DataFrame(columns=train_names + post_processing_names + ['iou', 'wrong_count'])

for i, split in tqdm(enumerate(kf.split(all_lectures))):
    print(f"###################### Split No. {i}")
    other_lectures = [all_lectures[index] for index in split[0]]
    test_lectures = [all_lectures[index] for index in split[1]]
    
    # Model selection
    df_all = model_selection(other_lectures, train_names, post_processing_names, filtered_by, split_by)    
    df_all['wrong_count'] = df_all['wrong_count'].astype(int)
    best_params = df_all.groupby(train_names + post_processing_names).mean().sort_values(by=['wrong_count', 'iou']).iloc[0].name
    best_params = convert_params(best_params)
    best_params = dict(zip(train_names + post_processing_names, best_params))
    test_filtered_by = partial(filtered_by, used=all_lectures)
    test_split_by = partial(split_by, validation=test_lectures)
    
    
    best_detector = FastAIScreenDetector(filtered_by=test_filtered_by, valid_func=test_split_by)
    best_detector.train(**best_params)
    
    test_frames = [frame for frame in all_frames if test_split_by(frame.pathname)]
    actuals = [actual_detector.detect(frame) for frame in test_frames]
    preds = [best_detector.detect(frame) for frame in test_frames]
    
    wrong_count, ious, _ = evaluate(actuals, preds)
    iou_score = np.nanmean(ious)
    wrong_count = len(wrong_count)
    df_best_models.loc[len(df_best_models)] = tuple(best_params.values()) + (iou_score, wrong_count)
    df_best_models.to_csv('cross_validation_results_frame_wise.csv', index=False)
    
df_best_models.to_csv('cross_validation_results_frame_wise.csv', index=False)

0it [00:00, ?it/s]
  0%|          | 0/1 [00:00<?, ?it/s]

###################### Split No. 0



100%|██████████| 1/1 [00:28<00:00, 28.77s/it]
1it [00:36, 36.63s/it]
  0%|          | 0/1 [00:00<?, ?it/s]

###################### Split No. 1



100%|██████████| 1/1 [00:29<00:00, 29.53s/it]
2it [01:14, 36.99s/it]
  0%|          | 0/1 [00:00<?, ?it/s]

...


100%|██████████| 1/1 [00:36<00:00, 36.15s/it]
3it [01:57, 38.90s/it]
  0%|          | 0/1 [00:00<?, ?it/s]

###################### Split No. 3



100%|██████████| 1/1 [00:32<00:00, 32.14s/it]
4it [02:36, 38.95s/it]
  0%|          | 0/1 [00:00<?, ?it/s]

###################### Split No. 4



100%|██████████| 1/1 [00:34<00:00, 34.68s/it]
5it [03:18, 39.61s/it]
